In [ ]:
from stream_dataset import HuggingfaceStreamDataset, TokenizedDatasetWrapper, collate_fn

stream_dataset = HuggingfaceStreamDataset('ZelaAI/minipile_test')


In [ ]:
for item in stream_dataset:
    item['text']


In [ ]:
import time

for item in TokenizedDatasetWrapper(HuggingfaceStreamDataset("ZelaAI/minipile_test")):
    item

In [ ]:
from stream_dataset import HuggingfaceStreamDataset, TokenizedDatasetWrapper, collate_fn

from torch.utils.data import DataLoader
dataset = TokenizedDatasetWrapper(HuggingfaceStreamDataset("ZelaAI/minipile_test", skip_to=5000))


dataloader = DataLoader(dataset, batch_size=2, num_workers=2, collate_fn=collate_fn)

In [ ]:
for item in dataloader:
    item

In [ ]:
# Goal is to allow for multiprocessing so we can dynamically load and process data easily as it comes in rather than loading all at once.

In [ ]:
# class SequenceStreamDataset(IterableDataset):
#     def __init__(self, streaming_dataset, skip_first_n=0, insert_dense_tokens=0, tokenizer=None):
#         print(f'Dataset skipping first {skip_first_n} items', flush=True)
#         self.streaming_dataset = streaming_dataset.skip(skip_first_n)
#         self.insert_dense_tokens = insert_dense_tokens
#         self.tokenizer = tokenizer

#     def __iter__(self):
#         for item in self.streaming_dataset:
#             sequences = []
#             for seq in item['packed_sequences']:
#                 context_ids = seq['context_ids']
#                 completion_ids = seq['completion_ids']
                
#                 if self.insert_dense_tokens > 0:
#                     num_to_insert = random.randint(1, self.insert_dense_tokens)

#                     # enforce never more than 1/3 of the sequence is dense tokens
#                     num_to_insert = min(num_to_insert, len(completion_ids)//3)
                    
#                     # shorten completion_ids to make room for dense tokens
#                     completion_ids = completion_ids[:-num_to_insert]

#                     # perform insertions
#                     for i in range(num_to_insert):
#                         index = random.randint(0, len(completion_ids))
#                         completion_ids.insert(index, DENSE_TOKEN_ID)
                
#                 sequences.append(Sequence(
#                     0,
#                     torch.tensor(context_ids, dtype=torch.long),
#                     0,
#                     [torch.tensor(completion_ids, dtype=torch.long)],
#                 ))
#             yield sequences